<a href="https://colab.research.google.com/github/eisbetterthanpi/pytorch/blob/main/translation_transformer_down.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title setup

# https://pytorch.org/tutorials/beginner/translation_transformer.html
# https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/c64c91cf87c13c0e83586b8e66e4d74e/translation_transformer.ipynb

# https://github.com/pytorch/data
%pip install portalocker
%pip install torchdata

# Create source and target language tokenizer. Make sure to install the dependencies.
!pip install -U torchdata
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm


In [2]:
# @title data

from torchtext.datasets import multi30k, Multi30k
# modify the URLs for the dataset since the links to the original dataset are broken https://github.com/pytorch/text/issues/1756#issuecomment-1163664163
multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

from torchtext.data.utils import get_tokenizer
de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')
en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')


UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3 # unknown, pad, bigining, end of sentence
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

from torchtext.vocab import build_vocab_from_iterator
train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

de_tokens = [de_tokenizer(data_sample[0]) for data_sample in train_iter]
en_tokens = [en_tokenizer(data_sample[1]) for data_sample in train_iter]

de_vocab = build_vocab_from_iterator(de_tokens, min_freq=1, specials=special_symbols, special_first=True)
en_vocab = build_vocab_from_iterator(en_tokens, min_freq=1, specials=special_symbols, special_first=True)
de_vocab.set_default_index(UNK_IDX)
en_vocab.set_default_index(UNK_IDX)

import torch

def de_transform(o):
    o=de_tokenizer(o)
    o=de_vocab(o)
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))

def en_transform(o):
    o=en_tokenizer(o)
    o=en_vocab(o)
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))


from torch.nn.utils.rnn import pad_sequence
# function to collate data samples into batch tensors
def collate_fn(batch): # convert a batch of raw strings into batch tensors
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(de_transform(src_sample.rstrip("\n")))
        tgt_batch.append(en_transform(tgt_sample.rstrip("\n")))
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    # src_batch = pad_sequence(src_batch, batch_first=True, padding_value=PAD_IDX)
    # tgt_batch = pad_sequence(tgt_batch, batch_first=True, padding_value=PAD_IDX)
    return src_batch, tgt_batch


torch.manual_seed(0)

train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
val_iter = Multi30k(split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
batch_size = 256 # 128
train_loader = torch.utils.data.DataLoader(train_iter, batch_size=batch_size, collate_fn=collate_fn)
val_loader = torch.utils.data.DataLoader(val_iter, batch_size=batch_size, collate_fn=collate_fn)

# vocab_transform = {SRC_LANGUAGE:de_vocab, TGT_LANGUAGE:en_vocab}
# text_transform = {SRC_LANGUAGE:de_transform, TGT_LANGUAGE:en_transform}


/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [3]:
# @title test
trainiter = iter(train_iter)
x = next(trainiter)
src_batch, tgt_batch = [], []

# src_sample, tgt_sample = next(trainiter)
# for src_sample, tgt_sample in x:
for i, (src_sample, tgt_sample) in enumerate(train_iter):
    # src_batch.append(de_transform(src_sample.rstrip("\n")))

    # o=src_sample.rstrip("\n")
    o=tgt_sample.rstrip("\n")
    print(o)
    # o=de_tokenizer(o)
    o=en_tokenizer(o)
    print(o)
    # o=de_vocab(o)
    o=en_vocab(o)
    print(o)
    o=torch.cat((torch.tensor([BOS_IDX]), torch.tensor(o), torch.tensor([EOS_IDX])))
    print(o)

    # src_batch.append(de_transform(src_sample.rstrip("\n")))
    # tgt_batch.append(en_transform(tgt_sample.rstrip("\n")))
    tgt_batch.append(o)
    if i >=3: break

# src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
# tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
tgt_batch = pad_sequence(tgt_batch, batch_first=True, padding_value=PAD_IDX)


print("#########")
for tgt_tokens in tgt_batch:
    out = " ".join(en_vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")
    print(out)


Two young, White males are outside near many bushes.
['Two', 'young', ',', 'White', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']
[19, 25, 15, 1169, 808, 17, 57, 84, 336, 1339, 5]
tensor([   2,   19,   25,   15, 1169,  808,   17,   57,   84,  336, 1339,    5,
           3])
Several men in hard hats are operating a giant pulley system.
['Several', 'men', 'in', 'hard', 'hats', 'are', 'operating', 'a', 'giant', 'pulley', 'system', '.']
[165, 36, 7, 335, 287, 17, 1224, 4, 758, 4496, 2957, 5]
tensor([   2,  165,   36,    7,  335,  287,   17, 1224,    4,  758, 4496, 2957,
           5,    3])
A little girl climbing into a wooden playhouse.
['A', 'little', 'girl', 'climbing', 'into', 'a', 'wooden', 'playhouse', '.']
[6, 61, 33, 232, 71, 4, 253, 4460, 5]
tensor([   2,    6,   61,   33,  232,   71,    4,  253, 4460,    5,    3])
A man in a blue shirt is standing on a ladder cleaning a window.
['A', 'man', 'in', 'a', 'blue', 'shirt', 'is', 'standing', 'on', 'a', 'ladder', 'cleaning',

In [4]:


# def collate_fn(batch): # convert a batch of raw strings into batch tensors
#     src_batch, tgt_batch = [], []
#     for src_sample, tgt_sample in batch:
#         src_batch.append(de_transform(src_sample.rstrip("\n")))
#         tgt_batch.append(en_transform(tgt_sample.rstrip("\n")))
#     src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
#     tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
#     # src_batch = pad_sequence(src_batch, batch_first=True, padding_value=PAD_IDX)
#     # tgt_batch = pad_sequence(tgt_batch, batch_first=True, padding_value=PAD_IDX)
#     return src_batch, tgt_batch


# batch_size = 128
# train_loader = torch.utils.data.DataLoader(train_iter, batch_size=batch_size, collate_fn=collate_fn)
# # val_loader = torch.utils.data.DataLoader(val_iter, batch_size=batch_size, collate_fn=collate_fn)


# def generate_square_subsequent_mask(sz):
#     mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
#     # mask = (torch.tril(torch.ones((sz, sz), device=device)) == 1)
#     mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
#     return mask

# def create_mask(src, tgt):
#     src_seq_len = src.shape[0]
#     tgt_seq_len = tgt.shape[0]
#     tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
#     src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)
#     src_padding_mask = (src == PAD_IDX).transpose(0, 1)
#     tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
#     return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

# loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
# # model = Transformer(num_encoder_layers, num_decoder_layers, emb_size, nhead, src_vocab_size, tgt_vocab_size, dim_feedforward).to(device)

# # with torch.no_grad
# for src, tgt in train_loader:
#     src = src.to(device)
#     tgt = tgt.to(device)
#     tgt_input = tgt[:-1, :]
#     src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
#     print(src, tgt_input, src_mask, tgt_mask)
#     logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
#     optimizer.zero_grad()
#     tgt_out = tgt[1:, :]
#     loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
#     break



In [5]:
# @title model
import torch
import torch.nn as nn
import math
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class PositionalEncoding(nn.Module):
    def __init__(self, emb_size, dropout, maxlen = 5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(dropout)
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1) # .reshape(-1, 1)
        pos_emb = torch.zeros((maxlen, emb_size))
        pos_emb[:, 0::2] = torch.sin(pos * den) # PE(pos, 2i) = sin(pos/1000^(2i/dim_model))
        pos_emb[:, 1::2] = torch.cos(pos * den) # PE(pos, 2i + 1) = cos(pos/1000^(2i/dim_model))
        pos_emb = pos_emb.unsqueeze(-2)
        self.register_buffer('pos_emb', pos_emb) # register as buffer so optimizer wont update it

    def forward(self, token_emb):
        return self.dropout(token_emb + self.pos_emb[:token_emb.size(0), :])


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)


class Transformer(nn.Module):
    def __init__(self, num_encoder_layers, num_decoder_layers, emb_size, nhead, src_vocab_size, tgt_vocab_size, dim_feedforward = 512, dropout = 0.1):
        super(Transformer, self).__init__()
        self.emb_size = emb_size
        self.src_tok_emb = nn.Embedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = nn.Embedding(tgt_vocab_size, emb_size)
        self.pos_enc = PositionalEncoding(emb_size, dropout=dropout)
        self.transformer = nn.Transformer(d_model=emb_size, nhead=nhead, num_encoder_layers=num_encoder_layers, num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward, dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)

        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)

    def forward(self, src, tgt, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask):
        src_emb = self.pos_enc(self.src_tok_emb(src))
        tgt_emb = self.pos_enc(self.tgt_tok_emb(tgt))

        # src = self.src_tok_emb(src.long()) * math.sqrt(self.emb_size) # https://datascience.stackexchange.com/questions/87906/transformer-model-why-are-word-embeddings-scaled-before-adding-positional-encod
        # src_emb = self.pos_enc(src)
        # tgt = self.src_tok_emb(tgt.long()) * math.sqrt(self.emb_size)
        # tgt_emb = self.pos_enc(tgt)

        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src, src_mask):
        return self.transformer.encoder(self.pos_enc(self.src_tok_emb(src)), src_mask)

        src = self.src_tok_emb(src.long()) * math.sqrt(self.emb_size)
        src_emb = self.pos_enc(src)
        return self.transformer.encoder(src_emb, src_mask)

    def decode(self, tgt, memory, tgt_mask):
        return self.transformer.decoder(self.pos_enc(self.tgt_tok_emb(tgt)), memory, tgt_mask)

        tgt = self.tgt_tok_emb(tgt.long()) * math.sqrt(self.emb_size)
        tgt_emb = self.pos_enc(tgt)
        return self.transformer.decoder(tgt_emb, memory, tgt_mask)


def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(device)
    src_mask = src_mask.to(device)
    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)
    for i in range(max_len-1):
        memory = memory.to(device)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys


src_vocab_size = len(de_vocab)
tgt_vocab_size = len(en_vocab)

emb_size = 512
nhead = 8
dim_feedforward = 512
num_encoder_layers = 3
num_decoder_layers = 3

transformer = Transformer(num_encoder_layers, num_decoder_layers, emb_size, nhead, src_vocab_size, tgt_vocab_size, dim_feedforward)

# for p in transformer.parameters():
#     print(p)
#     if p.dim() > 1:
#         nn.init.xavier_uniform_(p)

transformer = transformer.to(device)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
# @title Transformer small?
# class TokenEmbedding(nn.Module):
#     def __init__(self, vocab_size, emb_size):
#         super(TokenEmbedding, self).__init__()
#         self.embedding = nn.Embedding(vocab_size, emb_size)
#         self.emb_size = emb_size

#     def forward(self, tokens):
#         # return self.embedding(tokens.long())
#         return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# class Transformer(nn.Module):
#     def __init__(self, src_vocab_size, emb_size):
#         super(Transformer, self).__init__()
#         self.emb_size = emb_size
#         self.src_tok_emb1 = nn.Embedding(src_vocab_size, emb_size)
#         self.src_tok_emb2 = TokenEmbedding(src_vocab_size, emb_size)

#     def forward(self, tokens):
#         return self.src_tok_emb1(tokens) * math.sqrt(self.emb_size)
#         # return self.src_tok_emb2(tokens)

# transformer = Transformer(src_vocab_size, emb_size)

# for name, parameter in transformer.named_parameters():
#     print(name, parameter.dim())

# # src_tok_emb1.weight 2
# # src_tok_emb2.embedding.weight 2


In [7]:

# src = torch.randint(0,100,(33,128))
# src_tok_emb = nn.Embedding(src_vocab_size, emb_size)
# print(src_tok_emb(src) * math.sqrt(emb_size))
# src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
# print(src_tok_emb(src))


In [8]:
# @title train eval

# subsequent word mask that will prevent the model from looking into the future words when making predictions.
# also need masks to hide source and target padding token
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    # mask = (torch.tril(torch.ones((sz, sz), device=device)) == 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]
    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)
    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

# def train_epoch(dataloader, model, loss_fn, optimizer, scheduler=None, verbose=True):
def train_epoch(dataloader, model, loss_fn, optimizer):
    model.train()
    losses = 0
    for src, tgt in dataloader:
        src = src.to(device)
        tgt = tgt.to(device)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        optimizer.zero_grad()
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        optimizer.step()
        losses += loss.item()
    return losses / len(list(dataloader))

# def evaluate(dataloader, model, loss_fn, verbose=True):
def evaluate(dataloader, model, loss_fn):
    model.eval()
    losses = 0
    for src, tgt in dataloader:
        src = src.to(device)
        tgt = tgt.to(device)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()
    return losses / len(list(dataloader))

# actual function to translate input sentence into target language
def translate(model, src_sentence):
    model.eval()
    src = de_transform(src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(en_vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))


 china interacts shortly shortly shortly shortly shortly shortly shortly shortly shortly shortly shortly shortly shortly


In [ ]:
# @title wwwwwwwwwwwwww

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.001, betas=(0.9, 0.98), eps=1e-9) # lr=0.0001

import time
epochs = 18

for epoch in range(epochs):
    start_time = time.time()
    train_loss = train_epoch(train_loader, transformer, loss_fn, optimizer)
    end_time = time.time()
    val_loss = evaluate(val_loader, transformer, loss_fn)
    print((f"Epoch: {epoch+1}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))
    print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))


/usr/local/lib/python3.10/dist-packages/torch/utils/data/datapipes/iter/combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5076: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Epoch: 1, Train loss: 6.438, Val loss: 5.088, Epoch time = 44.948s
 A man in a in a in a in a 
Epoch: 2, Train loss: 4.789, Val loss: 4.395, Epoch time = 44.993s
 A group of people in a group of a group of a group of a
Epoch: 3, Train loss: 4.297, Val loss: 4.024, Epoch time = 46.102s
 A group of people are standing in a group of people . 
Epoch: 4, Train loss: 3.996, Val loss: 3.784, Epoch time = 45.868s
 A group of people are standing in front of a group of a group of
Epoch: 5, Train loss: 3.771, Val loss: 3.593, Epoch time = 46.206s
 A group of people standing in front of a group of people . 
Epoch: 6, Train loss: 3.595, Val loss: 3.446, Epoch time = 46.481s
 A group of people standing in front of a group of people . 
Epoch: 7, Train loss: 3.452, Val loss: 3.340, Epoch time = 46.442s
 A group of people standing in front of a building . 
Epoch: 8, Train loss: 3.326, Val loss: 3.241, Epoch time = 46.600s
 A group of people standing in front of a building . 
Epoch: 9, Train loss: 3.219

In [ ]:
for p in transformer.parameters():
    print(p)
    # if p.dim() > 1:
        # nn.init.xavier_uniform_(p)

In [ ]:
# @title inference

print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))
